In [9]:
import os
import numpy as np
import geopandas as gpd
import time
import shapely
import pandas as pd

from sampling_handler.sampling import grid
from sampling_handler.misc import py_helpers

import ee
ee.Initialize()

In [10]:
start=time.time()

aoi = ee.FeatureCollection("USDOS/LSIB/2017").filter(ee.Filter.Or(
    ee.Filter.stringContains('COUNTRY_NA', 'Congo'),
    ee.Filter.stringContains('COUNTRY_NA', 'Equatorial Guinea'),
    ee.Filter.stringContains('COUNTRY_NA', 'Gabon'),
    ee.Filter.stringContains('COUNTRY_NA', 'Central African'),
    ee.Filter.stringContains('COUNTRY_NA', 'Cameroon'),
)).union(100)


def convert_3d_to_2d(geom):
    return shapely.wkb.loads(shapely.wkb.dumps(geom, output_dimension=2))

def parallel_hexgrid(
        aoi,
        resolution,                      # refers to the resolution of the grid
        sampling_strategy='systematic',    # choices are 'random' or 'systematic'
        centroid_crs="ESRI:54017",         # defines in which projection your centroids are created from the hexagons
        outcrs='EPSG:4326',
        projection="ISEA3H",
        grid_only=False
):

    
    base_resolution = np.ceil(resolution/2)
    base_hexs = grid.hexagonal_grid(
        aoi, base_resolution, projection, grid_only=True
    )
    print(len(base_hexs))
    aoi_gdf = py_helpers.read_any_aoi_to_single_row_gdf(aoi)
    base_hexs['geometry'] = [
        geom.buffer(0.1).intersection(aoi_gdf.geometry)[0] 
        for geom in convert_to2d(base_hexs.geometry)
    ]

    dggrid_args = []
    results = []
    for i, row in base_hexs.iterrows():
        
        dggrid_args.append([
            gpd.GeoDataFrame([base_hexs.loc[i]]).set_crs('EPSG:4326'), 
            resolution, sampling_strategy, centroid_crs, outcrs, projection
        ])
        
        
    results = py_helpers.run_in_parallel(
        grid.hexagonal_grid, dggrid_args, os.cpu_count(), 'processes'
    )
        
    
    dfs = []
    for r in results:
        dfs.append(r[0])

    hexs = pd.concat(dfs)
    hexs.drop_duplicates('name', inplace=True)
    
    dfs = []
    for r in results:
        dfs.append(r[1])

    centroids = pd.concat(dfs)
    centroids.drop_duplicates('name', inplace=True)
    print(len(centroids))
    # add point id
    print("Adding a unique point ID...")
    centroids["point_id"] = [i for i in range(len(centroids))]
    print("Remove points outside AOI...")
    #centroids = centroids[centroids.geometry.within(aoi_gdf.geometry.unary_union)]
    return hexs, centroids

hexs, grids = parallel_hexgrid(aoi, 16)
py_helpers.timer(start, 'Hex grid done in')
print('Saving files')

grids.to_file('/home/sepal-user/cafi_hex16_samples.gpkg', driver='GPKG')
py_helpers.timer(start, 'File saved in')
hexs.to_file('/home/sepal-user/cafi_hex16.gpkg', driver='GPKG')

INFO: Reading AOI.
INFO: Creating hexagonal grid...
INFO: Time elapsed: 0:00:36.721857
608
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Reading AOI.
INFO: Creating hexagonal grid...
INFO: Creating hexagonal grid...
INFO: Creating hexagonal grid...
INFO: Creating hexagonal grid...
INFO: Creating hexagonal grid...
INFO: Creating hexagonal grid...
INFO: Creating hexagonal gr